## Lab 16 — Transformer “Sanity Tests” 

In this lab, we will use a real open-source Transformer (via **HuggingFace Transformers**) as a black box and run a set of **basic tests** to understand what is happening inside a modern LLM pipeline. Instead of training from scratch, we focus on **observability**: tokenization behavior, embedding geometry, and next-token prediction.

You will:
- Load a pretrained tokenizer + causal language model (e.g., **Qwen2.5**).
- Inspect **tokenization outputs** (tokens, token IDs, and how spaces are handled).
- Extract **input embeddings** and run a small **embedding similarity** experiment across words from different semantic categories.
- Implement a simple generation loop to compare **greedy decoding** vs **sampling with temperature**.
- Probe the model by printing the **top-k next token probabilities** for a given prompt, and interpret what those probabilities mean.

By the end, you should be able to explain:  
(1) why tokenization details matter, (2) what embeddings represent geometrically, and (3) how a Transformer turns a prompt into a probability distribution over the next token.

In [1]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
import numpy as np

## Obtaining a DeepSeek API Key

To call the DeepSeek API, you first need to obtain a personal **API key**.

1. Visit the DeepSeek official website:  
   https://platform.deepseek.com/

2. Sign up for an account (or log in if you already have one).

3. Go to the **API / Developer** section of the dashboard.

4. Create a new API key and copy it.

5. Paste the key into your notebook or script:
   ```python
   api_key = "YOUR_API_KEY_HERE"


In [2]:
api_key = ""
url = "https://api.deepseek.com/v1/chat/completions"
headers = {"Authorization": f"Bearer {api_key}"}

In [3]:
C_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model_name = "Qwen/Qwen2.5-1.5B" 
#model_name = "Qwen/Qwen2.5-0.5B"          
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#model = AutoModel.from_pretrained(model_name) 

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained( 
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

`torch_dtype` is deprecated! Use `dtype` instead!


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

model = model.to(device)

Using Device: cuda
GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Memory: 8.0 GB


In [6]:

print("Tokenizer:", type(tokenizer).__name__)
print("Vocab size:", tokenizer.vocab_size)
print("Special token:")
print(" - CLS:", tokenizer.cls_token)
print(" - SEP:", tokenizer.sep_token) 
print(" - PAD:", tokenizer.pad_token)
print(" - UNK:", tokenizer.unk_token)

Tokenizer: Qwen2TokenizerFast
Vocab size: 151643
Special token:
 - CLS: None
 - SEP: None
 - PAD: <|endoftext|>
 - UNK: None


In [7]:
test_texts = "All happy families are alike; each unhappy family is unhappy in its own way."
tokens = tokenizer.tokenize(test_texts)
token_ids = tokenizer.encode(test_texts)
print(f"\nOriginal: {test_texts}")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids[:30]}...")
print(f"toke num: {len(tokens)}")


Original: All happy families are alike; each unhappy family is unhappy in its own way.
Tokens: ['All', 'Ġhappy', 'Ġfamilies', 'Ġare', 'Ġalike', ';', 'Ġeach', 'Ġunhappy', 'Ġfamily', 'Ġis', 'Ġunhappy', 'Ġin', 'Ġits', 'Ġown', 'Ġway', '.']
Token IDs: [2403, 6247, 8521, 525, 25992, 26, 1817, 42151, 2997, 374, 42151, 304, 1181, 1828, 1616, 13]...
toke num: 16


In [8]:
def explain_space_handling():
    examples = [
        "hello world",  
        "hello  world", 
        "hello",         
        " hello",     
    ]
    
    for text in examples:
        tokens = tokenizer.tokenize(text)
        print(f"'{text}' → {tokens}")
        for token in tokens:
            if 'Ġ' in token:
                print(f" Note that  '{token}' has a space before it")
                
explain_space_handling()

'hello world' → ['hello', 'Ġworld']
 Note that  'Ġworld' has a space before it
'hello  world' → ['hello', 'Ġ', 'Ġworld']
 Note that  'Ġ' has a space before it
 Note that  'Ġworld' has a space before it
'hello' → ['hello']
' hello' → ['Ġhello']
 Note that  'Ġhello' has a space before it


In [9]:
embedding_layer = model.get_input_embeddings()
embedding_layer

Embedding(151936, 1536)

In [10]:
import torch
import torch.nn.functional as F

device = next(model.parameters()).device

def get_word_embedding(word, model, tokenizer):
    embedding_layer = model.get_input_embeddings()
    token_ids = tokenizer.encode(word, add_special_tokens=False)

    token_ids = torch.tensor(token_ids, device=device)

    with torch.no_grad():
        embeddings = embedding_layer(token_ids)   # [num_tokens, dim]
        return embeddings.mean(dim=0)            

def cosine_similarity_torch(vec1, vec2):
    return F.cosine_similarity(
        vec1.unsqueeze(0), 
        vec2.unsqueeze(0), 
        dim=1
    ).item()

words = ["cat", "dog", "lion", "wind", "rain", "snow", "run", "walk", "jump"]

print("🔍 Similarity matrix")
print("=" * 120)
print(" " * 12 + "".join([f"{word:>10}" for word in words]))


word_embeddings = {
    word: get_word_embedding(word, model, tokenizer)
    for word in words
}

for word1 in words:
    print(f"{word1:>12}: ", end="")
    for word2 in words:
        sim = cosine_similarity_torch(
            word_embeddings[word1],
            word_embeddings[word2]
        )

        if sim > 0.2:
            print(f"\033[92m{sim:>10.3f}\033[0m", end="")  
        elif sim < 0.1:
            print(f"\033[91m{sim:>10.3f}\033[0m", end="")  
        else:
            print(f"{sim:>10.3f}", end="")
    print()


🔍 Similarity matrix
                   cat       dog      lion      wind      rain      snow       run      walk      jump
         cat:      1.000     0.248     0.097     0.073     0.013     0.002     0.068     0.078     0.059
         dog:      0.248     1.000     0.191     0.078     0.047     0.113     0.054     0.117     0.038
        lion:      0.097     0.191     1.000     0.068     0.141     0.169     0.023     0.089     0.083
        wind:      0.073     0.078     0.068     1.000     0.153     0.109     0.073     0.165     0.098
        rain:      0.013     0.047     0.141     0.153     1.000     0.247     0.129     0.089     0.003
        snow:      0.002     0.113     0.169     0.109     0.247     1.000     0.034     0.086     0.081
         run:      0.068     0.054     0.023     0.073     0.129     0.034     1.000     0.153     0.090
        walk:      0.078     0.117     0.089     0.165     0.089     0.086     0.153     1.001     0.160
        jump:      0.059     0.038   

In [11]:
def robust_generate(prompt, model, tokenizer, max_new_tokens=50, entropy=0.5):

    device = model.device
    
    print(f"Input: '{prompt}'")
    print(f"Entropy: {entropy} ")
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs.input_ids
    
    generated_tokens = []
    
    for i in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
            

            if hasattr(outputs, 'logits'):
                logits = outputs.logits
            else:
                logits = outputs.last_hidden_state
            
            next_token_logits = logits[:, -1, :]
            

            if entropy == 0.0:
                next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
                strategy = "Greedy"
                
            else:

                temperature = 0.1 + entropy * 1.9 
                next_token_logits = next_token_logits / temperature
                probs = torch.softmax(next_token_logits, dim=-1)
                next_token_id = torch.multinomial(probs, num_samples=1)
                strategy = f"temperature(t={temperature:.1f})"
            
            new_token = tokenizer.decode(next_token_id[0], skip_special_tokens=True)
            generated_tokens.append(new_token)
            
            print(f"Token {i+1}: '{new_token}' ({strategy})")
            
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            
            # Stopping Criteria
            if next_token_id.item() == tokenizer.eos_token_id:
                break
            if new_token in ['\n', '.', '!', '?', '。', '！', '？']:
                break
    
    generated_text = prompt + ''.join(generated_tokens)
    print(f"Test: {generated_text}")
    return generated_text

In [12]:
def ask_question(question, model, tokenizer, max_answer_tokens=50, entropy=0.5):
    

    prompt = f"Question：{question}\n"
    print(f"🤔 question: {question}")

    full_response = robust_generate(prompt, model, tokenizer, max_new_tokens=max_answer_tokens, entropy=0.5)
    answer = full_response.replace(prompt, "").strip()
    
    print(f"\n🎯 Final answer: {answer}")
    print(f"📊 Length: {len(answer)} characters")
    
    return answer

In [13]:
question = "孔子是谁"
answer = ask_question(question, model, tokenizer, max_answer_tokens=50, entropy=0.2)

🤔 question: 孔子是谁
Input: 'Question：孔子是谁
'
Entropy: 0.5 


C:\Users\manyo\.conda\envs\atari\lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Token 1: '在' (temperature(t=1.1))
Token 2: '百家' (temperature(t=1.1))
Token 3: '争' (temperature(t=1.1))
Token 4: '鸣' (temperature(t=1.1))
Token 5: '的时代' (temperature(t=1.1))
Token 6: '里' (temperature(t=1.1))
Token 7: '孔子' (temperature(t=1.1))
Token 8: '还' (temperature(t=1.1))
Token 9: '算是' (temperature(t=1.1))
Token 10: '大' (temperature(t=1.1))
Token 11: '智慧' (temperature(t=1.1))
Token 12: '的' (temperature(t=1.1))
Token 13: '代表' (temperature(t=1.1))
Token 14: '，' (temperature(t=1.1))
Token 15: '后' (temperature(t=1.1))
Token 16: '世' (temperature(t=1.1))
Token 17: '有' (temperature(t=1.1))
Token 18: '婴' (temperature(t=1.1))
Token 19: '儒' (temperature(t=1.1))
Token 20: '先生' (temperature(t=1.1))
Token 21: '下' (temperature(t=1.1))
Token 22: '移' (temperature(t=1.1))
Token 23: '岁的' (temperature(t=1.1))
Token 24: '他' (temperature(t=1.1))
Token 25: '几乎' (temperature(t=1.1))
Token 26: '已' (temperature(t=1.1))
Token 27: '游' (temperature(t=1.1))
Token 28: '世' (temperature(t=1.1))
Token 29: '恩' (temp

In [15]:
def show_next_token_probabilities(prompt, model, tokenizer, top_k=20):

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
        if hasattr(outputs, 'logits'):
            logits = outputs.logits
        else:
            logits = outputs.last_hidden_state
        
        next_token_logits = logits[:, -1, :]
        
        # Calculate probability
        probs = torch.softmax(next_token_logits, dim=-1)
        
        # Obtain the top_k most likely token
        top_probs, top_indices = torch.topk(probs, top_k)
        
        print(f"🔍 Promppt: '{prompt}'")
        print(f"📊 Top {top_k} predictions for the next token:\n")
        
        for i, (prob, idx) in enumerate(zip(top_probs[0], top_indices[0])):
            token_text = tokenizer.decode([idx])

            display_text = repr(token_text)[1:-1]  # 去掉引号
            
            print(f"{i+1:2d}. '{display_text:10s}' (ID: {idx:5d}) - Probability: {prob.item():.4f}")


current_prompt = "问题：孔子是谁\n孔子是我国古代的大思想家，"
show_next_token_probabilities(current_prompt, model, tokenizer, top_k=100)

🔍 Promppt: '问题：孔子是谁
孔子是我国古代的大思想家，'
📊 Top 100 predictions for the next token:

 1. '大         ' (ID: 26288) - Probability: 0.4290
 2. '儒家        ' (ID: 114358) - Probability: 0.1371
 3. '也是        ' (ID: 100000) - Probability: 0.0832
 4. '教育        ' (ID: 99460) - Probability: 0.0435
 5. '他         ' (ID: 42411) - Probability: 0.0363
 6. '是         ' (ID: 20412) - Probability: 0.0262
 7. '他是        ' (ID: 104878) - Probability: 0.0196
 8. '政治        ' (ID: 101091) - Probability: 0.0169
 9. '我国        ' (ID: 101055) - Probability: 0.0147
10. '也         ' (ID: 74763) - Probability: 0.0113
11. '同时也是      ' (ID: 111733) - Probability: 0.0104
12. '春秋        ' (ID: 109385) - Probability: 0.0098
13. '又是        ' (ID: 104458) - Probability: 0.0090
14. '是我国       ' (ID: 108659) - Probability: 0.0088
15. '伟大的       ' (ID: 107792) - Probability: 0.0079
16. '他的        ' (ID: 100648) - Probability: 0.0077
17. '被         ' (ID: 99250) - Probability: 0.0056
18. '著名        ' (ID: 102280) - Probability: